In [ ]:
from matplotlib import pyplot as plt
import japanize_matplotlib
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression

In [ ]:
# データの読み込み
df = pd.read_csv('https://raw.githubusercontent.com/AMLab-Amsterdam/CEVAE/master/datasets/IHDP/csv/ihdp_npci_10.csv', header = None)
df.columns = ['T', 'y_factual', 'y_cfactual', 'mu0', 'mu1'] + [f'X_{i}' for i in range(1, 26)]
df.head()

In [ ]:
# 潜在反応変数から観測変数を作成し、本来は観測できない変数を削除
df['Y'] = df['y_factual']
df.drop(['y_factual', 'y_cfactual', 'mu0', 'mu1'], axis=1, inplace=True)
df.head()

In [ ]:
# T=1のユニットのYの平均値とT=0のユニットのYの平均値の差を計算

Y_1_mean = df[df['T'] == 1]['Y'].mean()
Y_0_mean = df[df['T'] == 0]['Y'].mean()
diff = Y_1_mean - Y_0_mean

print(f'T=1のユニットのYの平均値: {Y_1_mean}')
print(f'T=0のユニットのYの平均値: {Y_0_mean}')
print(f'T=1のユニットのYの平均値とT=0のユニットのYの平均値の差: {diff}')

In [ ]:
# 傾向スコアの推定

X = df.drop(['T', 'Y'], axis=1).values
T = df['T'].values

# ロジスティック回帰モデルを用いて傾向スコアを推定
model = LogisticRegression(penalty=None)
model.fit(X, T)
ps = model.predict_proba(X)[:, 1]

# 推定された傾向スコアをデータフレームに追加
df['ps'] = ps

df.tail()

In [ ]:
# 対照群と処置群で傾向スコアの分布を比較

plt.hist(df[df['T'] == 1]['ps'], bins=20, alpha=0.5, density=True, label='処置群')
plt.hist(df[df['T'] == 0]['ps'], bins=20, alpha=0.5, density=True, label='対照群')
plt.xlabel('傾向スコア')
plt.ylabel('密度')
plt.legend()
plt.show();

In [ ]:
# マッチング推定により平均処置効果を推定

# 処置群と統制群の傾向スコアが近いユニット同士をマッチング
df_1 = df[df['T'] == 1].copy()
df_0 = df[df['T'] == 0].copy()

matched_rows = []

for i, row in df_1.iterrows():
    dist = np.abs(df_0['ps'] - row['ps'])
    min_idx = dist.idxmin()
    
    # 処置群と統制群を1行にまとめる
    matched_pair = {
        'T1_index': i,                     # 処置群のインデックス
        'T1_ps': row['ps'],                # 処置群の傾向スコア
        'T1_outcome': row['Y'],      # 処置群のアウトカム
        'T0_index': min_idx,               # 統制群のインデックス
        'T0_ps': df_0.loc[min_idx, 'ps'],  # 統制群の傾向スコア
        'T0_outcome': df_0.loc[min_idx, 'Y']  # 統制群のアウトカム
    }
    matched_rows.append(matched_pair)

# データフレームに変換
df_match = pd.DataFrame(matched_rows)

df_match.tail()


In [ ]:
# マッチング推定量による推定値の計算
matching_estimate = df_match['T1_outcome'].mean() - df_match['T0_outcome'].mean()

print(f'マッチング推定による平均処置効果の推定値: {matching_estimate}')

In [ ]:
# 逆確率重み付け推定により平均処置効果を推定

# 逆確率重み付け推定
ipw_estimate = (np.sum(df['T'] * df['Y'] / df['ps']) - np.sum((1 - df['T']) * df['Y'] / (1 - df['ps']))) / len(df)

print(f'逆確率重み付け推定による平均処置効果の推定値: {ipw_estimate}')

In [ ]:
# 重回帰分析により平均処置効果を推定

# 重回帰分析
Y = df['Y'].values
T_X = np.concatenate([T.reshape(-1, 1), X], axis=1)

model = LinearRegression()
model.fit(T_X, Y)
coef = model.coef_[0]

print(f'重回帰分析による平均処置効果の推定値: {coef}')

In [ ]:
# 重回帰分析のベイズ推定により平均処置効果の事後分布を求める

# 事前分布の設定
prior_beta_mean = 0
prior_beta_s = 0.01
prior_s_e_a = 1
prior_s_e_b = 50

# gibbs sampling
np.random.seed(0)
X = T_X
y = Y
n = X.shape[0]
p = X.shape[1]

# 初期値
beta = np.zeros(p)
s_e = 1
n_iter = 10000
beta_samples = np.zeros((n_iter, p))
sig2_e_samples = np.zeros(n_iter)

for i in range(n_iter):
    # βのサンプリング
    beta_cov = np.linalg.inv(s_e * X.T @ X + prior_beta_s * np.eye(p) )
    beta_mean = beta_cov @ (s_e * X.T @ y + prior_beta_s * prior_beta_mean)
    beta = np.random.multivariate_normal(beta_mean, beta_cov)
    beta_samples[i] = beta

    # s_eのサンプリング
    s_e_a_new = prior_s_e_a + n/2
    s_e_b_new = prior_s_e_b + 0.5 * (y - X @ beta).T @ (y - X @ beta)
    s_e = np.random.gamma(s_e_a_new, 1/s_e_b_new)
    sig2_e_samples[i] = 1./s_e

In [ ]:
# 平均処置効果の事後分布の描画（事後サンプルのヒストグラム）
plt.hist(beta_samples[:, 0], bins=50, density=True, color='orange')
plt.xlabel(r'$\alpha$');

In [ ]:
# 平均処置効果の事後平均と95%信用区間を計算
lower = np.percentile(beta_samples[:, 0], 2.5)
upper = np.percentile(beta_samples[:, 0], 97.5)

print(f'事後平均による平均処置効果の推定値: {beta_samples[:, 0].mean()}')
print(f'平均処置効果の95%信用区間: ({lower}, {upper})')